In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aarshm","key":"e2beb4b63aedab6eb1ff9afbd8d20a02"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d tushar5harma/plant-village-dataset-updated

100% 1.00G/1.00G [00:09<00:00, 145MB/s]
100% 1.00G/1.00G [00:09<00:00, 118MB/s]


In [5]:
# @title Default title text
!unzip -o plant-village-dataset-updated.zip




Streaming output truncated to the last 5000 lines.
  inflating: Tomato/Train/Septoria Leaf Spot/a3938779-9d9f-4e45-a397-7b5abd13f6ba___JR_Sept.L.S 2700.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a4467299-5237-45f5-bb9b-9fd69f7a2b1a___Matt.S_CG 7719.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a45b192c-f31e-4ca1-bf4a-10ff1060e658___Matt.S_CG 7512.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a45bce54-4c25-4d5d-a9a9-fbc76436844c___Matt.S_CG 6523.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a46ff722-2186-4645-9018-01e8e20cfa2a___Matt.S_CG 6511.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a4880706-e191-43ff-a5a7-bd0f0c03aa2d___Keller.St_CG 1851.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a494e2ac-9dbe-47e5-9963-3e99f8eb9d2e___Matt.S_CG 6062.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a49c7f28-4b67-4d95-b79b-26a52dc8b9e5___Matt.S_CG 7583.JPG  
  inflating: Tomato/Train/Septoria Leaf Spot/a5bf2855-754e-4661-86b1-a86f0974cd30___JR_Sept.L.S 2574.JPG

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks
import matplotlib.pyplot as plt
from IPython.display import HTML

In [7]:
# Set your dataset path and parameters
data_path = "/content/Corn (Maize)/Train"  # Update this path
batch_size = 32
target_size = (224, 224)
num_epochs = 10


In [8]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Image_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split = 0.2
)


train_generator = Image_datagen.flow_from_directory(
    data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset = 'training'
)


validation_generator = Image_datagen.flow_from_directory(
    data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset = 'validation'
)

Found 5855 images belonging to 4 classes.
Found 1461 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'Apple Scab': 0, 'Black Rot': 1, 'Cedar Apple Rust': 2, 'Healthy': 3}

In [9]:
print("Training samples:", train_generator.samples)
print("Batch size:", train_generator.batch_size)

# Calculate steps per epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size
print("Calculated steps per epoch:", steps_per_epoch)

Training samples: 5855
Batch size: 32
Calculated steps per epoch: 182


In [12]:
class_names = list(train_generator.class_indices.keys())
class_names

['Cercospora Leaf Spot', 'Common Rust ', 'Healthy', 'Northern Leaf Blight']

In [ ]:
len(data_path)

21

In [ ]:
train_size = 0.8
len(data_path)*train_size

16.8

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Set your dataset path and parameters
data_path = "/content/Corn (Maize)/Train"
batch_size = 32
target_size = (224, 224)
num_epochs = 30

# Reverting to more aggressive data augmentation
Image_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Data generators
train_generator = Image_datagen.flow_from_directory(
    data_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = Image_datagen.flow_from_directory(
    data_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint(filepath='CornTransfer.h5', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=0.00001)

Found 5855 images belonging to 4 classes.
Found 1461 images belonging to 4 classes.


In [13]:
import tensorflow as tf
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the pre-trained InceptionaResNetV2 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


219055592/219055592 [==============================] - 1s 0us/step


In [14]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add a global average pooling layer
x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
predictions = Dense(4, activation='softmax')(x)  # Assuming you have 4 classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [15]:

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Use the same data generators and training process as before
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

Epoch 1/30
182/182 [==============================] - ETA: 0s - loss: 0.3997 - accuracy: 0.8623

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


182/182 [==============================] - 155s 743ms/step - loss: 0.3997 - accuracy: 0.8623 - val_loss: 0.2439 - val_accuracy: 0.9208 - lr: 5.0000e-05
Epoch 2/30
182/182 [==============================] - 111s 610ms/step - loss: 0.2056 - accuracy: 0.9294 - val_loss: 0.1963 - val_accuracy: 0.9271 - lr: 5.0000e-05
Epoch 3/30
182/182 [==============================] - 131s 721ms/step - loss: 0.1697 - accuracy: 0.9396 - val_loss: 0.1772 - val_accuracy: 0.9375 - lr: 5.0000e-05
Epoch 4/30
182/182 [==============================] - 113s 622ms/step - loss: 0.1576 - accuracy: 0.9416 - val_loss: 0.1634 - val_accuracy: 0.9438 - lr: 5.0000e-05
Epoch 5/30
182/182 [==============================] - 129s 711ms/step - loss: 0.1424 - accuracy: 0.9521 - val_loss: 0.1417 - val_accuracy: 0.9500 - lr: 5.0000e-05
Epoch 6/30
182/182 [==============================] - 133s 733ms/step - loss: 0.1291 - accuracy: 0.9521 - val_loss: 0.1332 - val_accuracy: 0.9535 - lr: 5.0000e-05
Epoch 7/30
182/182 [=============

In [ ]:
history = model.fit(train_generator, steps_per_epoch=100, epochs=10, validation_data=validation_generator, validation_steps=50)


Epoch 1/10
 10/100 [==>...........................] - ETA: 57s - loss: 0.0833 - accuracy: 0.9749

In [ ]:
files.download("CornTransfer.h5")


In [ ]:
test_generator = Image_datagen.flow_from_directory(
    data_path,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Typically set to False for evaluation
)

In [ ]:
from tensorflow.keras.models import load_model

# Load the transfer learning model
transfer_model = load_model("GrapeTransfer.h5")  # Replace with the actual path to your saved model

# Assuming you have already defined your test_generator
loss_transfer, accuracy_transfer = transfer_model.evaluate(test_generator)

print(f'Transfer Learning Model - Loss: {loss_transfer}, Accuracy: {accuracy_transfer}')


In [ ]:
# For your regular CNN model
loss_regular, accuracy_regular = model.evaluate(test_generator)

print(f'Regular CNN Model - Loss: {loss_regular}, Accuracy: {accuracy_regular}')




In [ ]:
import pandas as pd

data = {
    'Model': ['Regular CNN', 'Transfer Learning'],
    'Accuracy': [accuracy_regular, accuracy_transfer],
    'Loss': [loss_regular, loss_transfer]
}

df = pd.DataFrame(data)
print(df)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels = ['Accuracy', 'Loss']
regular_metrics = [accuracy_regular, loss_regular]
transfer_metrics = [accuracy_transfer, loss_transfer]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, regular_metrics, width, label='Regular CNN')
rects2 = ax.bar(x + width/2, transfer_metrics, width, label='Transfer Learning')

ax.set_ylabel('Scores')
ax.set_title('Scores by Model and Metric')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()
